# Notebook Imports

In [2]:

from pathlib import Path

import pandas as pd
import numpy as np

# Constants

In [96]:
TRAINING_PATH = Path("SpamData/02_Training")
TESTING_PATH = Path("SpamData/03_Testing")
# Create the path if not ixist
TRAINING_PATH.mkdir(parents=True, exist_ok=True)
TESTING_PATH.mkdir(parents=True, exist_ok=True)

TOKEN_ALL_PROB_FILE = str(TESTING_PATH / 'prob-all-tokens.txt')
TOKEN_SPAM_PROB_FILE = str(TESTING_PATH / 'prob-spam.txt')
TOKEN_HAM_PROB_FILE = str(TESTING_PATH / 'prob-ham.txt')

TEST_FEATURE_MATRIX = str(TESTING_PATH / 'test-features.txt')
TEST_TARGET_FILE = str(TESTING_PATH / 'test-target.txt')

TRAINING_DATA_FILE = str(TRAINING_PATH / 'train-data.txt')
TEST_DATA_FILE = str(TRAINING_PATH / 'test-data.txt')

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ',dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ',dtype=int)

In [6]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [7]:
sparse_train_data[-5:]

array([[5795, 2035,    0,    2],
       [5795, 2040,    0,    1],
       [5795, 2272,    0,    1],
       [5795, 2342,    0,    1],
       [5795, 2370,    0,    1]])

In [8]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258360
Nr of rows in test file 117776


In [10]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in test file 1724


## How to create an Empty DataFrame

In [11]:
column_names = ['DOC_ID'] + ['CATEGORY'] +  list(range(0, VOCAB_SIZE))
column_names[:5]


['DOC_ID', 'CATEGORY', 0, 1, 2]

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [14]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)

In [19]:
full_train_data.fillna(value=0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a  Full Matric from a Sparse Matrix

In [24]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of the occurance of word in the sparse matrix. Default: 4th column
    
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] +  list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [26]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [29]:
full_train_data.CATEGORY.size

4013

In [30]:
full_train_data.CATEGORY.sum()

1248

In [32]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.310989284824321


## Total number of Words / Tokens

In [34]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] # tokens subset
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
email_lengths = full_train_features.sum(axis=1) # sum up the columns, left to right
email_lengths.shape


(4013,)

In [37]:
total_wc = email_lengths.sum()
total_wc

429241

# Number of Tokens in Spam & Ham Emails

In [49]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [50]:
spam_wc = spam_lengths.sum()
spam_wc

176334

In [51]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [52]:
ham_wc = ham_lengths.sum()
ham_wc

252907

In [53]:
spam_wc + ham_wc - total_wc

0

In [55]:
print('Average nr of words in spam emails {:.0f}'.format( spam_wc / spam_lengths.shape[0]))

Average nr of words in spam emails 141


In [56]:
print('Average nr of words in nonspam emails {:.0f}'.format( ham_wc / ham_lengths.shape[0]))

Average nr of words in nonspam emails 91


## Summing the Tokens Occuring in Spam

In [57]:
full_train_features.shape

(4013, 2500)

In [60]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
train_spam_tokens.shape

(1248, 2500)

In [64]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [66]:
summed_spam_tokens.tail()

2495     3
2496     2
2497    17
2498    15
2499     2
dtype: int64

## Summing the Tokens Occuring in Non-Spam

In [68]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.tail()

2495    27
2496    35
2497    20
2498    15
2499    28
dtype: int64

# Probability of Viagra occurring in an email, given that the email is spam

$$ P(Spam \; | \; Viagra) = \frac {P(Viagra \; | \; Spam)} {P(Viagra)} \times P(Spam) $$

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [78]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam.loc[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
5    0.006140
dtype: float64

In [79]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is NonSpam

In [80]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_ham.loc[:5]

0    0.021472
1    0.010141
2    0.008007
3    0.003673
4    0.006311
5    0.006288
dtype: float64

In [81]:
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that Token Occurs

In [82]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [95]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [97]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [98]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)